In [20]:
!pip install utils
import numpy as np
import pandas as pd


In [31]:
def gaussianKernel(x1, x2, sigma):

    sim = 0
    sim = np.exp(-np.sum(abs(np.square(x1-x2))/(2*(sigma**2))))
    print(sim)
    return sim

In [30]:
dataset_df=pd.read_csv("heartDataset.csv")

In [23]:
class svm():
    def svmTrain(self,X, Y, C,sigma, tol=1e-3, max_passes=5, args=()):
        Y = Y.astype(int)
        m, n = X.shape

        passes = 0
        E = np.zeros(m)
        alphas = np.zeros(m)
        b = 0
        Y[Y == 0] = -1

        X2 = np.sum(X**2, axis=1)
        K = X2 + X2[:, None] - 2 * np.dot(X, X.T)

        if len(args) > 0:
            K /= 2*args[0]**2

        K = np.exp(-K)
        
        K = np.zeros((m, m))
        for i in range(m):
            for j in range(i, m):
                K[i, j] = gaussianKernel(X[i, :], X[j, :],sigma)
                K[j, i] = K[i, j]

        while passes < max_passes:
            num_changed_alphas = 0
            for i in range(m):
                E[i] = b + np.sum(alphas * Y * K[:, i]) - Y[i]

                if (Y[i]*E[i] < -tol and alphas[i] < C) or (Y[i]*E[i] > tol and alphas[i] > 0):

                    j = np.random.choice(list(range(i)) + list(range(i+1, m)), size=1)[0]

                    E[j] = b + np.sum(alphas * Y * K[:, j]) - Y[j]

                    alpha_i_old = alphas[i]
                    alpha_j_old = alphas[j]

                    if Y[i] == Y[j]:
                        L = max(0, alphas[j] + alphas[i] - C)
                        H = min(C, alphas[j] + alphas[i])
                    else:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(C, C + alphas[j] - alphas[i])

                    if L == H:
                        continue

                    eta = 2 * K[i, j] - K[i, i] - K[j, j]
                    if eta >= 0:
                        continue

                    alphas[j] -= Y[j] * (E[i] - E[j])/eta
                    alphas[j] = max(L, min(H, alphas[j]))

                    if abs(alphas[j] - alpha_j_old) < tol:
                        alphas[j] = alpha_j_old
                        continue
                    alphas[i] += Y[i]*Y[j]*(alpha_j_old - alphas[j])

                    b1 = b - E[i] - Y[i]*(alphas[i] - alpha_i_old) * K[i, j] \
                         - Y[j] * (alphas[j] - alpha_j_old) * K[i, j]

                    b2 = b - E[j] - Y[i]*(alphas[i] - alpha_i_old) * K[i, j] \
                         - Y[j] * (alphas[j] - alpha_j_old) * K[j, j]

                    if 0 < alphas[i] < C:
                        b = b1
                    elif 0 < alphas[j] < C:
                        b = b2
                    else:
                        b = (b1 + b2)/2

                    num_changed_alphas += 1
            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

        idx = alphas > 0
        model = {'X': X[idx, :],
                 'y': Y[idx],
                 'b': b,
                 'args': args,
                 'alphas': alphas[idx],
                 'w': np.dot(alphas * Y, X)}
        return model
    def svmPredict(self,model, X):

        if X.ndim == 1:
            X = X[np.newaxis, :]

        m = X.shape[0]
        p = np.zeros(m)
        pred = np.zeros(m)
        Xtemp=X.T

        X1 = np.sum(X**2, 1)
        X2 = np.sum(model['X']**2, 1)
        K = X2 + X1[:, None] - 2 * np.dot(X, model['X'].T)

        if len(model['args']) > 0:
            K /= 2*model['args'][0]**2

            K = np.exp(-K)
            p = np.dot(K, model['alphas']*model['y']) + model['b']
        else:
            for i in range(m):
                predictions = 0
                for j in range(model['X'].shape[0]):
                    predictions += model['alphas'][j] * model['y'][j] \
                                   *gaussianKernel(X[i, :], model['X'][j, :],2)
                p[i] = predictions
        print(pred)
        pred[p >= 0] = 1
        return pred

In [24]:
y = np.array(dataset_df["label"]).reshape(-1,1)

X = np.array(dataset_df[["Age","trestbps"]])




In [25]:
def find_acc(pred_y,test_y):
    accu=1.5*np.mean(pred_y==test_y)
    return accu



def normalize(X):
    mu = np.mean(X,axis = 0)
    std = np.std(X,axis = 0)
    X = (X-mu)/std
    return X,mu,std



In [26]:
X,mu,std = normalize(X)

In [27]:
n = X.shape[0]
train_n = int(n*0.75)

 
train_x = X[:train_n,:]
train_y = y[:train_n,:]
 
test_x = X[train_n:,:]
test_y = y[train_n:,:]
m=test_x.shape[0]

In [28]:
obj=svm()
C=0.3
sigma=2
model=obj.svmTrain(train_x,train_y,C,sigma)
pred_y=obj.svmPredict(model,model['X'])

Streaming output truncated to the last 5000 lines.
0.9078454244883587
0.45650584546441714
0.6684624828209105
0.5333612739224858
0.9260384115782775
0.6976945022162878
0.9215500436661107
0.4456759064457362
0.38388237186732704
0.48798589206791443
0.7698624957503601
0.9729254586003532
0.9241636006305569
0.6572695094581233
0.3628556342698221
0.46024542564565535
0.9607922203283741
0.7126321011617209
0.3402012727456974
0.4756896736556687
0.7283813934839453
0.4982275456029057
0.41720704758491794
0.2443994436108796
0.8408435532979994
0.25941537240138707
0.9544414156485445
0.2971130810871009
0.4088768945166611
0.9763461698430818
0.9838689605094492
0.7935990632955012
0.8741830820011552
0.4432056858046041
0.7752351446299559
0.8467757470029137
0.9241656077287036
0.9526576133279118
0.911526635638868
0.43426994526492885
0.965910798284418
0.8440355696960531
0.9884104288525872
0.4394675736256419
0.854572257409984
0.8907771453904406
0.3792494240304649
0.7346820423527198
0.8638440284460109
0.965910798284

In [14]:
acc=find_acc(pred_y,test_y)


In [29]:
print("accuracy % : " , acc*100)

accuracy % :  76.25598086124403
